For our final project our group proproses a 

Our data product allows users to input an address and find the top six restaurants from the inputted form. Through investigative methodologies, we were able to discover that the way that Yelp returns its list of restaurant is very blackboxed--existence of reviews, reviews by 'elite' Yelp members, check-ins via smartphone, etc. We wanted to present to users a more transparent way of choosing businesses that they may want to go to without using Yelp's method of choosing bigger businesses. In order to determine the top six restaurants in an area, our group wrote an algorithm that ranks restaurants based on X,Y, AND Z. 

Our home page at pwnrs.herokuapp.com/yelp renders the form for users to search based on their desired location. Upon clicking the 'Find a Restaurant' button, a new page is rendered in which our top 6 restuarants are displayed in a grid with the accompanying image url that we parsed from the API. 

Below the restaurant grid, we graph the distribution of the ratings of restaurants in that specific location with a (()).

In [ ]:
# imports for our Flask Application
from flask import Flask
from flask import render_template
from flask_sqlalchemy import SQLAlchemy
from flask import request
from flask import redirect

import matplotlib
matplotlib.use('Agg')

import os
import scrapy
import api_helper
import urllib.parse as urlparse
import psycopg2

import sys                             # system module
import pandas as pd                    # data package
import matplotlib.pyplot as plt, mpld3        # graphics module
import numpy as np                     # foundation for Pandas

In [ ]:
# set directory names for our templating and static files
app = Flask(
    __name__,
    template_folder="templates",
    static_folder="static"
)

Below we have our Location Data Model which shows how we format our data in our database. JEREMY ADD MORE

In [ ]:
# config for DB schematics
app.config['SQLALCHEMY_DATABASE_URI'] = os.environ['DATABASE_URL']
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)


# Locaiton data model
class Location(db.Model):
    """docstring for Location."""
    id = db.Column(db.Integer, primary_key=True)
    location = db.Column(db.Text, nullable=False)

    def __init__(self, location):
        self.location = location

    def __repr__(self):
        return '<Location %s>' % self.location

    def __str__(self):
        return '%s' % self.location


In [ ]:

# 
@app.route('/')
def home():
    top_10 = scrapy.scrap()
    df = pd.DataFrame.from_dict(top_10, orient='index')
    fig = plt.figure()
    plt.bar(
        x=np.arange(len(df.index)),
        height=df[0],
        align='center',
        alpha=0.5,
        tick_label=df.index,
    )
    return render_template('index.html', data=mpld3.fig_to_html(fig))


# router to main page of our application
@app.route('/yelp', methods=['POST', 'GET'])
def yelp():
    
    #  if a post request is issued   
    if request.method == 'POST':
        if request.form:

            # get form data from user input
            address = request.form['address']
            city = request.form['city']
            state = request.form['state']
            zipcode = request.form['zip']

            # final address string
            final_add = address + ' ' + city + ' ' + state + ' ' + zipcode

            # strip off empty spaces
            final_add = final_add.strip()

            # send a GET request to the YELP api with the locaiton
            response = api_helper.get_food_at_location(final_add)

            # add to database is Response from API is valid
            if response != None and response.status_code == 200:

                # add address input by user to DB
                add_location_to_db(final_add)

                # parse response to json
                final_stuff = response.json()
                businesses = final_stuff['businesses']

                # get top 6 businesses to display
                top_six = get_n_businesses(6, businesses)

                # render html page w/ jinja templating
                return render_template('yelp.html', top_six=top_six)
            return render_template('yelp.html')
    else:
        # render template if request is not POST        
        return render_template('yelp.html')


# helper function for getting n business from YELP response
def get_n_businesses(n, businesses):
    # array storage for businesses
    top_n = []
    
    # get name, url, img_url, rating, and price from each business
    for business in businesses[:n]:
        one_business = {}
        one_business['name'] = business['name']
        one_business['url'] = business['url']
        one_business['img_url'] = business['image_url']
        one_business['rating'] = business['rating']
        one_business['price'] = business['price']
        top_six.append(one_business)

    # return array of businesses
    return top_n

# helper function for adding location to DB
def add_location_to_db(address):
    location = Location(address)
    db.session.add(location)
    db.session.commit()

# get the locations with the most searches
def get_frequent_locations(num_locations):
    locations = db.session.query(Location.location, db.func.count(Location.location).label('Searches'))\
        .group_by(Location.location)\
        .order_by(db.desc('Searches'))\
        .limit(num_locations)\
        .all()
    return locations


# debugger for testing the application
if __name__ == '__main__':
    app.run(debug=True)
